Import necessary libraries.
For the CQM to work it is necessary to obtain the Solver API Token from https://cloud.dwavesys.com/leap/.

In [1]:
from dimod import ConstrainedQuadraticModel, Binary, Integer, SampleSet, BinaryArray
from dwave.system import LeapHybridCQMSampler
import numpy as np

Define the properties of the LRB problem

In [2]:
num_of_processors = 4
num_of_tasks = 16
tasks_duration = np.array([15.5, 15.5, 15.5, 15.5, 
                            38.1, 38.1, 38.1, 38.1, 
                            66.7, 66.7, 66.7, 66.7, 
                            67.4, 67.4, 67.4, 67.4])
max_number_of_moved = 3

In [3]:
original_assignment = [
    1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0, # processor 0 has tasks 0,1,2,3 (ids 0,1,2,3)
    0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0, # processor 1 has tasks 4,5,6,7 (ids 20,21,22,23)
    0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0, # processor 2 has tasks 8,9,10,11 (ids 40,41,42,43)
    0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1, # processor 3 has tasks 12,13,14,15 (ids 60,61,62,63)
]

In [4]:
def find_makespans(task_duration, assignment):
    time_on_processors = []
    for i in range(num_of_processors):
        time_on_processors.append(np.sum(np.multiply(task_duration,assignment[i*16:(i+1)*16])))
        
    return time_on_processors

In [5]:
makespans = find_makespans(tasks_duration, original_assignment)
makespans

[62.0, 152.4, 266.8, 269.6]

In [6]:
max_makespan = np.max(makespans)
avg_makespan = np.mean(makespans)
print(f"max {max_makespan}, avg {avg_makespan}")

max 269.6, avg 187.70000000000002


Use a [Constrained Quadratic Model](https://docs.ocean.dwavesys.com/en/stable/concepts/cqm.html) from D-Wave Ocean, and then define the necessary binary variables $x_{i,j}$ that will represent the new assignment of the tasks to the processors.

In [7]:
cqm = ConstrainedQuadraticModel()

In [8]:
new_assignment = BinaryArray([
                f'x{i}_{j}' for i in range(num_of_processors) for j in range(num_of_tasks)])

In [9]:
len(new_assignment) # in this encoding we need 64 binary variables

64

Objective function

min $\sum_{i=0}^{M-1} (avg - \sum_{j=0}^{N-1} t_j \cdot x_{i,j})^2 $, where $ avg = \frac{\sum_{j} t_j}{N}$

The closer the objective function to 0, the better the more equally distributed the tasks.

In [10]:
objective_function = 0

for i in range(num_of_processors):
    tmp_sum = 0
    for j in range(num_of_tasks):
        tmp_sum += tasks_duration[j] * new_assignment[num_of_tasks * i + j]
    objective_function += (avg_makespan - tmp_sum)**2

In [11]:
objective_function

BinaryQuadraticModel({'x0_0': -5578.450000000001, 'x0_1': -5578.450000000001, 'x0_2': -5578.450000000001, 'x0_3': -5578.450000000001, 'x0_4': -12851.130000000001, 'x0_5': -12851.130000000001, 'x0_6': -12851.130000000001, 'x0_7': -12851.130000000001, 'x0_8': -20590.290000000005, 'x0_9': -20590.290000000005, 'x0_10': -20590.290000000005, 'x0_11': -20590.290000000005, 'x0_12': -20759.2, 'x0_13': -20759.2, 'x0_14': -20759.2, 'x0_15': -20759.2, 'x1_0': -5578.450000000001, 'x1_1': -5578.450000000001, 'x1_2': -5578.450000000001, 'x1_3': -5578.450000000001, 'x1_4': -12851.130000000001, 'x1_5': -12851.130000000001, 'x1_6': -12851.130000000001, 'x1_7': -12851.130000000001, 'x1_8': -20590.290000000005, 'x1_9': -20590.290000000005, 'x1_10': -20590.290000000005, 'x1_11': -20590.290000000005, 'x1_12': -20759.2, 'x1_13': -20759.2, 'x1_14': -20759.2, 'x1_15': -20759.2, 'x2_0': -5578.450000000001, 'x2_1': -5578.450000000001, 'x2_2': -5578.450000000001, 'x2_3': -5578.450000000001, 'x2_4': -12851.1300000

In [12]:
cqm.set_objective(objective_function)

Constraints

In [13]:
# Migrate no more than k jobs
moved_in_total = 0
for j in range(num_of_tasks):
    is_this_task_moved = 1 #todo rename
    for i in range(num_of_processors):
        is_this_task_moved -= original_assignment[num_of_tasks*i + j] * new_assignment[num_of_tasks*i + j]
    moved_in_total += is_this_task_moved
    
cqm.add_constraint(moved_in_total <= max_number_of_moved)

'cc408e5'

In [15]:
# The assignment cannot be worse than the original one
for i in range(num_of_processors):
    equation_no_worse_assignment = 0
    for j in range(num_of_tasks):
        equation_no_worse_assignment += tasks_duration[j] * new_assignment[num_of_tasks*i + j]
    cqm.add_constraint(equation_no_worse_assignment <= max_makespan)

In [14]:
# Each task has to be assigned to exactly one processor
for j in range(num_of_tasks):
    equation_single_task = 0
    for i in range(num_of_processors):
        equation_single_task += new_assignment[num_of_tasks *i + j]
    cqm.add_constraint(equation_single_task == 1)

In [16]:
print(cqm)

Constrained quadratic model: 64 variables, 21 constraints, 736 biases

Objective
  140925.16000000003 - 5578.450000000001*Binary('x0_0') - 5578.450000000001*Binary('x0_1') - 5578.450000000001*Binary('x0_2') - 5578.450000000001*Binary('x0_3') - 12851.130000000001*Binary('x0_4') - 12851.130000000001*Binary('x0_5') - 12851.130000000001*Binary('x0_6') - 12851.130000000001*Binary('x0_7') - 20590.290000000005*Binary('x0_8') - 20590.290000000005*Binary('x0_9') - 20590.290000000005*Binary('x0_10') - 20590.290000000005*Binary('x0_11') - 20759.2*Binary('x0_12') - 20759.2*Binary('x0_13') - 20759.2*Binary('x0_14') - 20759.2*Binary('x0_15') - 5578.450000000001*Binary('x1_0') - 5578.450000000001*Binary('x1_1') - 5578.450000000001*Binary('x1_2') - 5578.450000000001*Binary('x1_3') - 12851.130000000001*Binary('x1_4') - 12851.130000000001*Binary('x1_5') - 12851.130000000001*Binary('x1_6') - 12851.130000000001*Binary('x1_7') - 20590.290000000005*Binary('x1_8') - 20590.290000000005*Binary('x1_9') - 20590.

In [17]:
print(cqm)

Constrained quadratic model: 64 variables, 21 constraints, 736 biases

Objective
  140925.16000000003 - 5578.450000000001*Binary('x0_0') - 5578.450000000001*Binary('x0_1') - 5578.450000000001*Binary('x0_2') - 5578.450000000001*Binary('x0_3') - 12851.130000000001*Binary('x0_4') - 12851.130000000001*Binary('x0_5') - 12851.130000000001*Binary('x0_6') - 12851.130000000001*Binary('x0_7') - 20590.290000000005*Binary('x0_8') - 20590.290000000005*Binary('x0_9') - 20590.290000000005*Binary('x0_10') - 20590.290000000005*Binary('x0_11') - 20759.2*Binary('x0_12') - 20759.2*Binary('x0_13') - 20759.2*Binary('x0_14') - 20759.2*Binary('x0_15') - 5578.450000000001*Binary('x1_0') - 5578.450000000001*Binary('x1_1') - 5578.450000000001*Binary('x1_2') - 5578.450000000001*Binary('x1_3') - 12851.130000000001*Binary('x1_4') - 12851.130000000001*Binary('x1_5') - 12851.130000000001*Binary('x1_6') - 12851.130000000001*Binary('x1_7') - 20590.290000000005*Binary('x1_8') - 20590.290000000005*Binary('x1_9') - 20590.

Use the Hybrid CQM Sampler/Solver

In [18]:
sampler = LeapHybridCQMSampler()
raw_sampleset = sampler.sample_cqm(cqm, time_limit=5)

From all the samples returned by the solver select only the ones which satisfy the constraints

In [19]:
feasible_sampleset = raw_sampleset.filter(lambda d: d.is_feasible)

In [20]:
feasible_sampleset

SampleSet(rec.array([([1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],  816.94, 1, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],  True),
           ([1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],  806.46, 1, [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True],  True),
           ([1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,

In [21]:
sample_id = 0
chosen_sample = feasible_sampleset.samples()[sample_id]

In [22]:
chosen_sample

{'x0_0': 1.0, 'x0_1': 0.0, 'x0_10': 0.0, 'x0_11': 0.0, 'x0_12': 0.0, 'x0_13': 0.0, 'x0_14': 1.0, 'x0_15': 0.0, 'x0_2': 1.0, 'x0_3': 1.0, 'x0_4': 0.0, 'x0_5': 0.0, 'x0_6': 0.0, 'x0_7': 0.0, 'x0_8': 1.0, 'x0_9': 0.0, 'x1_0': 0.0, 'x1_1': 1.0, 'x1_10': 0.0, 'x1_11': 0.0, 'x1_12': 0.0, 'x1_13': 0.0, 'x1_14': 0.0, 'x1_15': 0.0, 'x1_2': 0.0, 'x1_3': 0.0, 'x1_4': 1.0, 'x1_5': 1.0, 'x1_6': 1.0, 'x1_7': 1.0, 'x1_8': 0.0, 'x1_9': 0.0, 'x2_0': 0.0, 'x2_1': 0.0, 'x2_10': 1.0, 'x2_11': 1.0, 'x2_12': 0.0, 'x2_13': 0.0, 'x2_14': 0.0, 'x2_15': 0.0, 'x2_2': 0.0, 'x2_3': 0.0, 'x2_4': 0.0, 'x2_5': 0.0, 'x2_6': 0.0, 'x2_7': 0.0, 'x2_8': 0.0, 'x2_9': 1.0, 'x3_0': 0.0, 'x3_1': 0.0, 'x3_10': 0.0, 'x3_11': 0.0, 'x3_12': 1.0, 'x3_13': 1.0, 'x3_14': 0.0, 'x3_15': 1.0, 'x3_2': 0.0, 'x3_3': 0.0, 'x3_4': 0.0, 'x3_5': 0.0, 'x3_6': 0.0, 'x3_7': 0.0, 'x3_8': 0.0, 'x3_9': 0.0}

In [23]:
def get_new_assignment(num_of_processors, num_of_tasks, chosen_sample):
    result = []
    for i in range(num_of_processors):
        for j in range(num_of_tasks):
            result.append(int(chosen_sample[f'x{i}_{j}']))
    return result

In [24]:
result = get_new_assignment(num_of_processors, num_of_tasks, chosen_sample)

In [25]:
for i in range(num_of_processors):
    for j in range(num_of_tasks):
        print(result[num_of_tasks * i + j], end=',')
    print()

1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,
0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,
0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,
0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,


After load rebalancing

In [26]:
find_makespans(tasks_duration, result)

[180.60000000000002, 167.9, 200.10000000000002, 202.20000000000002]

Before load rebalancing

In [27]:
find_makespans(tasks_duration, original_assignment)

[62.0, 152.4, 266.8, 269.6]

In [28]:
for i in range(num_of_processors):
    for j in range(num_of_tasks):
        print(original_assignment[num_of_tasks * i + j], end=',')
    print()

1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,
0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,
0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,
0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,
